<a href="https://colab.research.google.com/github/liam-haas/CLPS0950_FinalProject/blob/main/Project_Testbed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install gradio


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.7/286.7 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.3/75.3 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 

In [ ]:
%pip install rdkit-pypi

from rdkit import Chem
from rdkit.Chem import *
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Descriptors
from rdkit.Chem import AllChem
from rdkit import DataStructs
from rdkit.Chem import PandasTools
from rdkit.Chem import RDKFingerprint

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 18.4 MB/s eta 0:00:00


In [32]:
import numpy as np

A = 'C=CC'
B = 'C=CC=C'

def index_fun(x):
  indices = [index for index in range(len(x)) if x.startswith('=',index)]
  # print(indices) # prints all occurances of double bonds in the SMILES string
  
  for n in range(0,len(indices)):

    a = indices[n] #testing to see if a phenyl is present
    #  try:
    #      x[a+6] != '1'
    #  except IndexError:
    #      continue
    #  return 
    
    if x[a-1] == 'C' and x[a+1] == 'C': # and x[a+6] != '1'
      

      print('Alkene found at position ', a)
    else:
      continue

  return indices

index_fun(A)

SyntaxError: ignored

[Gardio set-up](https://https://gradio.app/quickstart/)

In [ ]:


import pandas as pd
# pd.read_csv('Test SMILES sequences - Sheet1.csv')
d = {'IUPAC name':['ethene','prop-1-ene','3-methylpent-1-ene','1-phenylprop-2-ene','but-1,3-diene'],
     'SMILES sequence':['C=C','C=CC','C=CC(CC)C','C=CCC1=CC=CC=C1','C=CC=C']}
d['Alkenes'] = [index_fun(item) for item in d['SMILES sequence']] 
df = pd.DataFrame(data=d)
df

# d.update({'Alkene positions':''})

In [ ]:
import gradio as gr

def SMILEStoMol(x):
  return Chem.MolFromSmiles(x)

demo = gr.Interface(SMILEStoMol, inputs=gr.Textbox(lines=1, placeholder="Enter SMILES Sequence Here"), outputs="image")

demo.launch()

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

**New strategy below:** converting SMILES string to list and searching for alkenes.

In [61]:
A = 'C=CC'
B = 'C=CC=C'

def convert(string):
  str_as_list = []
  str_as_list[:0] = string
  return str_as_list

newA = convert(A)

if '=' in newA:
  i = A.index('=')
  print("Found an alkene at position", i)

print(i)

del newA[i:i+2]
print(newA)

newA.insert(i, 'C')
newA.insert(i+1, '(Br)')
print(newA)

print(''.join(newA))

def rxn1_fun(x, i):

  return

Found an alkene at position 1
1
['C', 'C']
['C', 'C', '(Br)', 'C']
CC(Br)C
